In [ ]:
"""
Created on Mon Mar 13 17:28 2023

Look at history and model files

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import pandas as pd
import glob

READ IN DATA

In [ ]:
inputpath_models = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/experiments/'
outputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_bi646/'


In [ ]:
mod_test = xr.open_dataset(inputpath_models + 'WHOLE/model_nn_medium_newbasic_wholedataset_TSextrap_normstd.h5', engine = 'h5netcdf')

In [ ]:
mod_test

In [ ]:
hist_test = pd.read_csv(inputpath_models + 'WHOLE/history_medium_newbasic_wholedataset_TSextrap_normstd.csv')

In [ ]:
hist_test['val_loss'].min()

In [ ]:
da_list = []
for mod_size in ['mini','small','medium','large','extra_large']:
    min_list = []
    nisf_list = []
    file_list = glob.glob(inputpath_models + 'CV_ISF/history_'+mod_size+'_newbasic_noisf*_notblock*_TSextrap_normstd.csv')
    for ff in file_list:
        hist_test = pd.read_csv(ff)
        min_loss = hist_test['val_loss'].min()
        kisf = int(ff.split('/')[-1].split('.')[0].split('isf')[1].split('_')[0])
        nisf_list.append(kisf)
        min_list.append(min_loss)

    min_da = xr.DataArray(data=np.array(min_list), dims=['Nisf']).assign_coords({'mod_size':mod_size,'Nisf': nisf_list})
    da_list.append(min_da)

all_rmse = xr.concat(da_list, dim='mod_size')

In [ ]:
int(ff.split('/')[-1].split('.')[0].split('isf')[1].split('_')[0])

In [ ]:
for mod_size in ['mini','small','medium','large','extra_large']:
    for kisf in all_rmse.Nisf:
        plt.scatter(all_rmse.sel(Nisf=kisf,mod_size=mod_size), mod_size, color='grey', alpha=0.2)
        #plt.text(all_rmse.sel(Nisf=kisf,mod_size=mod_size), mod_size, str(all_rmse.Nisf.sel(Nisf=kisf).values), color="red", fontsize=12)
plt.xlim(0,5)

In [ ]:
file_isf['isf_name'].sel(Nisf=52)

In [ ]:
file_isf = xr.open_dataset(outputpath_mask + 'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_1970.nc')

In [ ]:
file_isf['isf_name'].sel(Nisf=38)

In [ ]:
isf_names = file_isf['isf_name']

In [ ]:
plotted_var = all_rmse

x = np.arange(len(plotted_var.mod_size))
y = np.arange(len(plotted_var.Nisf))

fig, axs = plt.subplots(1, 1,figsize=((len(y)+1)/4,(len(x)+2)/2.75))
ax0 = axs.imshow(plotted_var.values, cmap=plt.cm.coolwarm, vmin=-100, vmax=100)

for j, dom in enumerate([50]):
    axs.yaxis.tick_right()
    axs.set_yticks(x)
    axs.yaxis.tick_right()

    axs.xaxis.tick_top()    
    axs.set_xticks(y)
    axs.set_xticklabels(labels=isf_names.sel(Nisf=plotted_var.Nisf).values, rotation=90)
    
